In [159]:
import pandas as pd
import re
import json
from typing import List
from sssom import Mapping, MappingSet
from IPython.core.display import HTML

pato_labels_file="../ontology/reports/pato.tsv"
uberon_labels_file="../ontology/reports/uberon.tsv"
efo_labels_file="../ontology/reports/efo.tsv"
oba_labels_file="../ontology/reports/oba.tsv"
cl_labels_file="../ontology/reports/cl.tsv"
go_labels_file="../ontology/reports/go.tsv"
vt_labels_file="../ontology/reports/vt.tsv"

# OBA Alignment Work

Make sure that you update the source data

```
sh run.sh make prepare_oba_alignment
```

We are doing three seperate things here:

1. Aligning EFO with OBA. This involves matching EFO classes to VT and OBA classes with SSSOM
2. Trying to patternise EFO classes by matching qualities and entities in its labels
3. Aligning VT with OBA by trying to patternise VT classes that have not been included in OBA so far


## Reading all data

The only input to this simple process is a table with labels and exact synonyms for all participating ontologies:

For matching potential DOSDP patterns:

* PATO (Qualities)
* UBERON (Entities)
* CL (Entities)
* GO (Entities)

For matching traits in general:

* VT
* EFO
* OBA

In [216]:
def read_labels(fn, prefix):
    labels = pd.read_csv(fn, sep="\t")
    labels.columns = ['id', 'predicate', 'value', 'type']
    labels = labels[['id', 'predicate', 'value']]
    labels['id']=labels['id'].str.replace('<http://purl.obolibrary.org/obo/','', regex=False)
    labels['id']=labels['id'].str.replace('<http://www.ebi.ac.uk/efo/','', regex=False)
    labels['id']=labels['id'].str.replace('>','', regex=False)
    labels['id']=labels['id'].str.replace('_',':', regex=False)
    labels['predicate']=labels['predicate'].str.replace('<http://www.w3.org/2000/01/rdf-schema#','rdfs:', regex=False)
    labels['predicate']=labels['predicate'].str.replace('<http://www.geneontology.org/formats/oboInOwl#','oboInOwl:', regex=False)
    labels['predicate']=labels['predicate'].str.replace('>','', regex=False)
    labels=labels[labels['id'].str.contains(prefix)]
    return labels

In [217]:
pato_labels=read_labels(pato_labels_file, "PATO")
pato_labels

,id,predicate,value
2,PATO:0000451,rdfs:label,obsolete pilosity value
5,PATO:0001519,rdfs:label,sound quality
14,PATO:0002215,rdfs:label,falciform
18,PATO:0000951,rdfs:label,purple
20,PATO:0000142,rdfs:label,obsolete substance
...,...,...,...
27144,PATO:0002045,oboInOwl:hasExactSynonym,dendriform
27176,PATO:0001624,oboInOwl:hasExactSynonym,having decreased function
27177,PATO:0001624,oboInOwl:hasExactSynonym,partial functionality
27178,PATO:0001624,oboInOwl:hasExactSynonym,low functionality


In [218]:
uberon_labels=read_labels(uberon_labels_file, "UBERON")
uberon_labels

,id,predicate,value
0,UBERON:0007232,rdfs:label,2 cell stage
1,UBERON:0007713,rdfs:label,fourth sacral spinal ganglion
2,UBERON:4300020,rdfs:label,anal fin basal cartilage
5,UBERON:0009657,rdfs:label,artery of lip
6,UBERON:0002370,rdfs:label,thymus
...,...,...,...
63342,UBERON:0008874,oboInOwl:hasExactSynonym,respiratory lobule
63343,UBERON:0008874,oboInOwl:hasExactSynonym,lobulus pulmonis primarius
63344,UBERON:0001999,oboInOwl:hasExactSynonym,Hyrtl's muscle
63345,UBERON:0001999,oboInOwl:hasExactSynonym,iliopsoas muscle


In [219]:
cl_labels=read_labels(cl_labels_file, "CL")
cl_labels

,id,predicate,value
6,CL:1000380,rdfs:label,type 1 vestibular sensory cell of epithelium o...
17,CL:0005021,rdfs:label,mesenchymal lymphangioblast
19,CL:0000397,rdfs:label,ganglion interneuron
20,CL:1000291,rdfs:label,myocyte of posterior internodal tract
21,CL:0000878,rdfs:label,central nervous system macrophage
...,...,...,...
61603,CL:0000935,oboInOwl:hasExactSynonym,"CD4-negative, CD8-negative, alpha-beta intraep..."
61604,CL:0000935,oboInOwl:hasExactSynonym,"CD4-negative, CD8-negative, alpha-beta intraep..."
61664,CL:0002506,oboInOwl:hasExactSynonym,DC.103+11b-.Lv
61707,CL:0002670,oboInOwl:hasExactSynonym,type I spiral ligament fibrocyte


In [164]:
go_labels=read_labels(go_labels_file, "GO")
go_labels

,id,predicate,value
0,GO:0071389,rdfs:label,cellular response to mineralocorticoid stimulus
1,GO:0007561,rdfs:label,imaginal disc eversion
2,GO:0051685,rdfs:label,maintenance of ER location
3,GO:0034275,rdfs:label,kynurenic acid metabolic process
4,GO:0060870,rdfs:label,cell wall disassembly involved in floral organ...
...,...,...,...
175363,GO:0015997,oboInOwl:hasExactSynonym,ubiquinone formation monooxygenase activity
175364,GO:0015997,oboInOwl:hasExactSynonym,ubiquinone anabolism monooxygenase activity
175365,GO:0015997,oboInOwl:hasExactSynonym,coenzyme Q biosynthesis monooxygenase activity
175366,GO:0015997,oboInOwl:hasExactSynonym,coenzyme Q biosynthetic process monooxygenase ...


In [165]:
vt_labels=read_labels(vt_labels_file, "VT")
vt_labels

,id,predicate,value
0,VT:0003477,rdfs:label,nerve fiber response trait
1,VT:0003799,rdfs:label,macrophage migration trait
2,VT:0005208,rdfs:label,iris stroma morphology trait
3,VT:0010765,rdfs:label,head and neck integrity trait
4,VT:0010508,rdfs:label,neurocranium mass
...,...,...,...
7143,VT:0002295,oboInOwl:hasExactSynonym,pulmonary circulation
7144,VT:0000953,oboInOwl:hasExactSynonym,oligoglia morphology trait
7145,VT:0005463,oboInOwl:hasExactSynonym,CD4+ T cell physiology trait
7146,VT:0005463,oboInOwl:hasExactSynonym,CD4+ T cell function


In [166]:
efo_labels=read_labels(efo_labels_file, "EFO")
efo_labels

,id,predicate,value
0,EFO:0010138,rdfs:label,nitrite measurement
1,EFO:0020165,rdfs:label,AT-rich interactive domain-containing protein ...
2,EFO:0008214,rdfs:label,lymphotactin measurement
3,EFO:0005273,rdfs:label,sleep depth
4,EFO:0010773,rdfs:label,CD5 measurement
...,...,...,...
4537,EFO:0005117,oboInOwl:hasExactSynonym,"Regulated on Activation, Normal T cell Express..."
4538,EFO:0009272,oboInOwl:hasExactSynonym,VCA-IgG seropositivity
4539,EFO:0009272,oboInOwl:hasExactSynonym,VCA seropositivity
4540,EFO:0010381,oboInOwl:hasExactSynonym,PC 36:3


In [167]:
oba_labels=read_labels(oba_labels_file, "OBA")
oba_labels

,id,predicate,value
0,OBA:0005640,rdfs:label,philtrum amount
11,OBA:VT0009921,rdfs:label,transitional stage T3 B cell morphology trait
15,OBA:VT0010453,rdfs:label,abdominal wall mass
22,OBA:VT0005208,rdfs:label,iris stroma morphology trait
24,OBA:VT0001148,rdfs:label,testes size trait
...,...,...,...
96244,OBA:0002099,oboInOwl:hasExactSynonym,amount of sensory perception of smell
96245,OBA:0003173,oboInOwl:hasExactSynonym,symmetry of ear
96247,OBA:0003002,oboInOwl:hasExactSynonym,2-D shape of phalanx of manus
96250,OBA:1000249,oboInOwl:hasExactSynonym,quality of basicranium


## EFO alignment

In the following, we will attempt an EFO alignment with OBA. At the same time, we will try to patternise EFO classes, with the goal of including these newly patternised EFO classes straight into OBA and mapping them back to EFO.

The process goes like this:

For all measurement terms T in EFO, 

1. Try to match T to OBA (for the purpose of having a straight mapping)
2. Try to match T to VT (for the purpose of having an indirect mapping we can use to link to OBA through VT-OBA alignment)
3. Try to match a PATO term.
4. If 3 was successful, we also try to match an UBERON, CL and/or GO term.
5. EFO-OBA and EFO-VT mappings are exported as SSSOM
6. EFO to EQ mappings are exported as a DOSDP pattern file with a bit of metadata

### Methods for the process

The central ideas in the mapping process are:

1. We remove noisy words like "trait" or "measurement" from our labels prior to alignment (we also do the usual preprocessing like lower casing etc)
2. For EFO-OBA/VT we take a naive process which attempts to only match _exactly_ the preprocessed labels
3. For the EFO-EQ patternisation we search for occurrences of E's in the label of the EFO term

In [197]:
# These are words we consider noise for the sake of this alignment process. Its probably worth adding more
stopwords = ["trait", "measurement"]

# Function that describes what we consider a "whole word" match in terms of regex. Probably
# Can be simplified.
def whole_word_regex(stopwords):
    stopwords_regex = []
    for stopword in stopwords:
        stopword = re.escape(stopword)
        stopwords_regex.append(f"[ .;,:]{stopword}[ .;,:]")
        stopwords_regex.append(f"[ ]{stopword}$")
        stopwords_regex.append(f"^{stopword}[ .;,:]")
        stopwords_regex.append(f"^{stopword}$")
    return stopwords_regex

# The label is prepared: lower cased, trimmed, stop words removed.
# Note this should probably do stemming, lemmatisation and _proper_ stop word removal (and, or, of) as well
def prepare_label(value, stopwords=stopwords):
    stopwords_regex = whole_word_regex(stopwords)                           
    if isinstance(value, str):
        for regex_value in stopwords_regex:
            value = re.sub(regex_value, "", value)
        value = value.lower()
        value = value.strip()
        return value
    else: 
        return ""


# Determines if the "value_to_match" is contained in label _as a whole word_. So active would not be 
# matched in a label which says inactive.
def whole_word_match(label, value_to_match, min_match_size=3):
    if value_to_match in label:
        whole_word_regexes = whole_word_regex([value_to_match,])
        for regex_value in whole_word_regexes:
            if(re.search(regex_value, label)):
                #print(f"Match found: {regex_value} in {label}")
                return True
    return False

def echo(string):
    #print(string)
    pass

# the value (probably an OBA, VT or EFO label) is matched against everything in the dataframe df. 
# strict_word_order=False would split the words of the label into a list, then sorting them, which allows
# matching words which are simply changed in order, like "cell count" vs "count of cell"
def get_matches(value, df, strict_word_order=True, exact_only=True, min_match_size=3):
    matches = []
    if not strict_word_order:
        # This sorts the words in the string before attempting to match
        value = " ".join(sorted(value.split(" ")))
    
    for index, row in df.iterrows():
        curie = row['id']
        predicate = row['predicate']
        label = prepare_label(row['value'])
        if len(label)>min_match_size:
            if not strict_word_order:
                label = " ".join(sorted(label.split(" ")))
            if label==value:
                matches.append({"match_string": label, "object_id": curie , "object_label": row['value'], 'object_match_field' : predicate, 'predicate_id': 'skos:exactMatch' })
            elif not exact_only and whole_word_match(value, label, min_match_size):
                matches.append({"match_string": label, "object_id": curie , "object_label": row['value'], 'object_match_field' : predicate, 'predicate_id': 'skos:relatedMatch' })
    
    return matches

### EFO Analysis

See summary of process above.

In [199]:
df = efo_labels.copy()
df = df.reset_index()  # make sure indexes pair with number of rows

mlist = []
mlist: List[Mapping] = []
    
dosdp_matches = []

for index, row in df.iterrows():
    curie = row['id']
    mdict = {}
    predicate = row['predicate']
    label = prepare_label(row['value'])
    mdict['subject_id']=curie
    mdict['subject_match_field']=predicate
    mdict['subject_label']=label
    
    echo("")
    echo("--------------------------------")
    echo(f"{curie}: {label}")
    
    echo("")
    echo("Matching OBA and VT")
    traits_oba = get_matches(label, oba_labels, strict_word_order=True, exact_only=True)
    traits_vt = get_matches(label, vt_labels, strict_word_order=True, exact_only=True)
    
    if traits_oba or traits_vt:
        echo("")
        echo("Outcomes")
        for matches in [traits_vt, traits_oba]:
            for matches_mdict in matches:
                echo(f"    {matches_mdict['object_id']} ({json.dumps(matches_mdict, sort_keys=True, indent=4)})")
                matches_mdict.update(mdict)
                matches_mdict['match_type']="HumanCurated"
                mlist.append(Mapping(**matches_mdict))
    
    echo("")
    echo("Matching PATO")
    qualities = get_matches(label, pato_labels, exact_only=False)
    
    if not qualities:
        echo("No quality matches, continuing")
        continue
    
    # Make Label wo quality reference
    label_wo_quality = label
    for q in qualities:
        label_wo_quality = label_wo_quality.replace(q['match_string'],"")
    label_wo_quality = label_wo_quality.strip()
    
    
    echo("")
    echo("Matching Uberon")
    anatomical_entities = get_matches(label_wo_quality, uberon_labels, strict_word_order=True, exact_only=False)
     
    echo("")
    echo("Matching CL")
    cell_types = get_matches(label_wo_quality, cl_labels, strict_word_order=True, exact_only=False)
    
    echo("")
    echo("Matching GO")
    bp_mf_ccs = [] #get_matches(label_wo_quality, go_labels, exact_only=False)
    
    
    for qmatches in qualities:
        pattern_dict = mdict.copy()
        pattern_dict['quality'] = qmatches['object_id']
        pattern_dict['quality_label'] = qmatches['object_label']
        for matches_list in [anatomical_entities, cell_types, bp_mf_ccs]:
            largest_e = []
            cache_es = {}
            echo(f"OL {ematches['object_label']}")
            for ematches in matches_list:
                process = True
                if len(largest_e)>0:
                    for e in largest_e:
                        echo(f"S: {e}")
                        if e in ematches['object_label']:
                            # Bigger one found
                            largest_e.remove(e)
                            process = True
                            break
                        elif ematches['object_label'] in e:
                            process = False
                            break
                                
                if process:
                    largest_e.append(ematches['object_label'])
                    pattern_dict['attribute'] = ematches['object_id']
                    pattern_dict['attribute_label'] = ematches['object_label']
                    np = pattern_dict.copy()
                    cache_es[ematches['object_label']]=np
            for e in largest_e:
                dosdp_matches.append(cache_es[e])


### Results

#### DOSDP matches found

In [223]:
dosdp_candidates = pd.DataFrame(dosdp_matches)
display(HTML(dosdp_candidates.to_html()))

,subject_id,subject_match_field,subject_label,quality,quality_label,attribute,attribute_label
0,EFO:0007719,rdfs:label,carotid artery external diameter,PATO:0001334,diameter,UBERON:0005396,carotid artery
1,EFO:0009230,rdfs:label,reticulocyte corpuscular hemoglobin distribution width,PATO:0000921,width,CL:0000558,reticulocyte
2,EFO:0009230,rdfs:label,reticulocyte corpuscular hemoglobin distribution width,PATO:0000060,distribution,CL:0000558,reticulocyte
3,EFO:0009238,rdfs:label,immature plasma cell count,PATO:0001501,immature,UBERON:0001969,plasma
4,EFO:0009238,rdfs:label,immature plasma cell count,PATO:0001501,immature,CL:0000786,plasma cell
5,EFO:0004767,rdfs:label,visceral:subcutaneous adipose tissue ratio,PATO:0001470,ratio,UBERON:0001013,adipose tissue
6,EFO:0004767,rdfs:label,visceral:subcutaneous adipose tissue ratio,PATO:0002438,subcutaneous,UBERON:0001013,adipose tissue
7,EFO:0005935,rdfs:label,overweight body mass index status,PATO:0000125,mass,UBERON:0000468,body
8,EFO:0007956,rdfs:label,monocyte:lymphocyte ratio,PATO:0001470,ratio,CL:0001054,monocyte
9,EFO:0010326,rdfs:label,precuneus cortex volume measurement@en,PATO:0000918,volume,UBERON:0006093,precuneus cortex


#### Number of qualities matched

In [215]:
df_quals = dosdp_candidates['quality_label'].value_counts().rename_axis('unique_values').reset_index(name='counts')
df_quals.head(10)

,unique_values,counts
0,volume,94
1,ratio,45
2,rate,26
3,density,23
4,thickness,22
5,white,20
6,distribution,17
7,width,16
8,size,13
9,pressure,12


#### Matches involving "Volume"

See https://github.com/obophenotype/bio-attribute-ontology/issues/97

In [222]:
df_vol=dosdp_candidates[dosdp_candidates['quality_label']=='volume']
display(HTML(df_vol.to_html()))

,subject_id,subject_match_field,subject_label,quality,quality_label,attribute,attribute_label
9,EFO:0010326,rdfs:label,precuneus cortex volume measurement@en,PATO:0000918,volume,UBERON:0006093,precuneus cortex
12,EFO:0009405,rdfs:label,parasubiculum volume,PATO:0000918,volume,UBERON:0004683,parasubiculum
18,EFO:0600045,rdfs:label,pancreas volume,PATO:0000918,volume,UBERON:3011040,pancreas
19,EFO:0010330,rdfs:label,superior parietal cortex volume measurement@en,PATO:0000918,volume,UBERON:0006094,superior parietal cortex
26,EFO:0010317,rdfs:label,paracentral lobule volume measurement@en,PATO:0000918,volume,UBERON:0035933,paracentral lobule
31,EFO:0006930,rdfs:label,brain volume,PATO:0000918,volume,UBERON:6110636,brain
34,EFO:0010335,rdfs:label,third ventricle volume measurement@en,PATO:0000918,volume,UBERON:0002286,third ventricle
39,EFO:0010604,rdfs:label,urine volume measurement@en,PATO:0000918,volume,UBERON:0001088,urine
42,EFO:0010304,rdfs:label,frontal pole volume measurement@en,PATO:0000918,volume,UBERON:0002795,frontal pole
47,EFO:0010321,rdfs:label,pars triangularis volume measurement@en,PATO:0000918,volume,UBERON:0002629,pars triangularis


### Prepareing EFO-OBA/VT mappings

Basically exporting them as SSSOM

In [207]:
from typing import List
from sssom import Mapping, MappingSet
from sssom.sssom_document import MappingSetDocument
from sssom.parsers import to_mapping_set_dataframe
from sssom.writers import write_table

prefix_map={
    'OBA': 'http://purl.obolibrary.org/obo/OBA_',
    'GO': 'http://purl.obolibrary.org/obo/OBA_',
    'UBERON': 'http://purl.obolibrary.org/obo/OBA_',
    'CL': 'http://purl.obolibrary.org/obo/OBA_',
    'VT': 'http://purl.obolibrary.org/obo/OBA_',
    'PATO': 'http://purl.obolibrary.org/obo/OBA_',
    'EFO': 'http://www.ebi.ac.uk/efo/EFO_',
    'oboInOwl': 'http://www.geneontology.org/formats/oboInOwl#',
    'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 
}

mapping_set_id = "https://w3id.org/sssom/commons/phenotype/efo2oba"
license = "https://github.com/callahantiff/OMOP2OBO/blob/master/LICENSE"
sssom_file_path = "../ontology/mappings/oba-all.sssom.tsv"


ms = MappingSet(mapping_set_id=mapping_set_id, license=license)

ms.mappings = mlist  # type:ignore
doc = MappingSetDocument(mapping_set=ms, prefix_map=prefix_map)
msdf = to_mapping_set_dataframe(doc)

f = open(sssom_file_path, "w")
write_table(msdf,f)
f.close()

,subject_id,subject_label,predicate_id,object_id,object_label,match_type,subject_match_field,object_match_field,match_string
0,EFO:0000714,survival,skos:exactMatch,VT:0005372,survival,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasExactSynonym,survival
1,EFO:0003923,bone mineral density,skos:exactMatch,VT:0005007,bone mineral density,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasRelatedSynonym,bone mineral density
2,EFO:0003923,bone density,skos:exactMatch,OBA:1000110,bone density,HumanCurated,rdfs:label,rdfs:label,bone density
3,EFO:0004301,blood viscosity,skos:exactMatch,VT:3000004,blood viscosity trait,HumanCurated,rdfs:label,rdfs:label,blood viscosity
4,EFO:0004301,blood viscosity,skos:exactMatch,OBA:VT3000004,blood viscosity trait,HumanCurated,rdfs:label,rdfs:label,blood viscosity
...,...,...,...,...,...,...,...,...,...
76,EFO:0010972,blood phosphate level,skos:exactMatch,VT:0001565,blood phosphate level,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasRelatedSynonym,blood phosphate level
77,EFO:0011043,neck circumference,skos:exactMatch,OBA:1000031,neck circumference,HumanCurated,rdfs:label,rdfs:label,neck circumference
78,EFO:0011043,neck circumference,skos:exactMatch,OBA:1000031,neck circumference trait,HumanCurated,rdfs:label,oboInOwl:hasExactSynonym,neck circumference
79,EFO:0600035,heart left atrium capacity,skos:exactMatch,VT:0004063,heart left atrium capacity,HumanCurated,oboInOwl:hasExactSynonym,rdfs:label,heart left atrium capacity


#### EFO-OBA/VT mappings

In [221]:
display(HTML(msdf.df.to_html()))

,subject_id,subject_label,predicate_id,object_id,object_label,match_type,subject_match_field,object_match_field,match_string
0,EFO:0000714,survival,skos:exactMatch,VT:0005372,survival,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasExactSynonym,survival
1,EFO:0003923,bone mineral density,skos:exactMatch,VT:0005007,bone mineral density,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasRelatedSynonym,bone mineral density
2,EFO:0003923,bone density,skos:exactMatch,OBA:1000110,bone density,HumanCurated,rdfs:label,rdfs:label,bone density
3,EFO:0004301,blood viscosity,skos:exactMatch,VT:3000004,blood viscosity trait,HumanCurated,rdfs:label,rdfs:label,blood viscosity
4,EFO:0004301,blood viscosity,skos:exactMatch,OBA:VT3000004,blood viscosity trait,HumanCurated,rdfs:label,rdfs:label,blood viscosity
5,EFO:0004305,erythrocyte count,skos:exactMatch,VT:0001586,erythrocyte count,HumanCurated,rdfs:label,oboInOwl:hasExactSynonym,erythrocyte count
6,EFO:0004305,erythrocyte number,skos:exactMatch,VT:0001586,erythrocyte number,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasExactSynonym,erythrocyte number
7,EFO:0004305,red blood cell count,skos:exactMatch,VT:0001586,red blood cell count,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasExactSynonym,red blood cell count
8,EFO:0004306,rbc count,skos:exactMatch,VT:0001586,RBC count,HumanCurated,oboInOwl:hasExactSynonym,oboInOwl:hasExactSynonym,rbc count
9,EFO:0004308,leukocyte count,skos:exactMatch,VT:0000217,leukocyte count,HumanCurated,rdfs:label,oboInOwl:hasExactSynonym,leukocyte count


## VT - OBA alignment